## Gradient Boosting Model on Satellite Data + previous ergot engineer features

- Gradient builds an additive model in a forward stage-wise fashion, which allows for the optimization of arbitrary differentiable loss functions. 

In [1]:
# dependencies

import pandas as pd
import sqlalchemy as sq
import sys, os
import pickle
from imblearn.combine import SMOTEENN, SMOTETomek
from xgboost import XGBClassifier
from sklearn.ensemble import (  # type: ignore
    GradientBoostingClassifier,
)
from imblearn.ensemble import (  # type: ignore
    RUSBoostClassifier,
)

from sklearn.metrics import (  # type: ignore
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)

sys.path.append("../../")
os.chdir("../../")
from ModelBuilderMethods import getConn, extractYears

In [2]:
# unlimited line output
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 500)

### <u>**Step 1**</u>: Data Selection

In this step, we would choose the particular data/table, pick attributes from existing tables. Further aggregation/feature engineer can be done here to support the point of the research.

Particular, for this notebook, we grab the following data and merge them (on year, district) into a single table:
- Satellite weather data
- ergot data (downgrade)
- ergot previous feature engineer

In [3]:
weatherSatQuery = sq.text(
    """
    SELECT * from dataset_cross_monthly_sat
"""
)

ergotPrevYearsAggQuery = sq.text(
    """
    SELECT year, district, 
    present_prev1, present_prev2, present_prev3,
    percnt_true_prev1, percnt_true_prev2, percnt_true_prev3 
    from agg_ergot_sample_v2
"""
)

ergotTargetQuery = sq.text(
    """
    SELECT year, district, downgrade from ergot_sample_feat_eng
"""
)

In [4]:
conn = getConn("./.env")

satelliteDf = pd.read_sql(weatherSatQuery, conn)
ergotPrevDf = pd.read_sql(ergotPrevYearsAggQuery, conn)
ergotTargetDf = pd.read_sql(ergotTargetQuery, conn)

conn.close()
del conn

In [5]:
# merge on year and district
tempdf = pd.merge(satelliteDf, ergotPrevDf, on=["year", "district"], how="left")
del satelliteDf
del ergotPrevDf

# merge on year and district
datasetDf = pd.merge(ergotTargetDf, tempdf, on=["year", "district"], how="left")
del ergotTargetDf
del tempdf

Then, the district number would be categorized using [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) techniques.

One-hot encoding techniques would take input, which is a categorical value. Then, the features are encoded using a one-hot (aka ‘one-of-K’ or ‘dummy’) encoding scheme. This creates a binary column for each category and returns a sparse matrix or dense array (depending on the sparse_output parameter).


In [6]:
# encode district
datasetDf["district"] = datasetDf["district"].astype("category")

temp = pd.get_dummies(datasetDf["district"], prefix="district", drop_first=True)
datasetDf = pd.concat([datasetDf, temp], axis=1)

datasetDf = datasetDf.drop(columns=["district"])
datasetDf["present_prev1"] = datasetDf["present_prev1"].astype("bool")
datasetDf["present_prev2"] = datasetDf["present_prev2"].astype("bool")
datasetDf["present_prev3"] = datasetDf["present_prev3"].astype("bool")

del temp

### <u>**Step 2**</u>: Splitting dataset

- We split the whole dataset into the train/test split. Particularly, split them by year (1995 - 2015 for training, 2016 - 2020 for testing) since this is a time series data.

In [7]:
# train 1995 - 2015 test 2016 - 2020
trainDf = extractYears(datasetDf, 1995, 2015)
testDf = extractYears(datasetDf, 2016, 2020)
del datasetDf

In [8]:
# drop year
trainDf = trainDf.drop(columns=["year"])
testDf = testDf.drop(columns=["year"])

### <u>**Step 3**</u>: [Balancing the dataset](https://imbalanced-learn.org/stable/)

- Our dataset is unbalanced and can lead to bias when training/testing. Balacing step would help to eliminate the bias of the dataset, thus provide more reliable results.

In [9]:
# pre balancing check
# print value counts downgrade
print(trainDf["downgrade"].value_counts())
print(testDf["downgrade"].value_counts())

downgrade
False    122202
True       2082
Name: count, dtype: int64
downgrade
False    26307
True      1016
Name: count, dtype: int64


In [10]:
# count nan
print(trainDf.isna().sum())
# set nan to 0
# trainDf = trainDf.fillna(0)

# drop nan
trainDf = trainDf.dropna()

downgrade                           0
1:min_dewpoint_temperature          0
1:min_temperature                   0
1:min_evaporation_from_bare_soil    0
1:min_skin_reservoir_content        0
                                   ..
district_4830                       0
district_4840                       0
district_4850                       0
district_4860                       0
district_4870                       0
Length: 693, dtype: int64


In [11]:
balancer = SMOTEENN(sampling_strategy=1, random_state=42)
balancedTrainDfX, balancedTrainDfY = balancer.fit_resample(
    trainDf.drop(columns="downgrade"), trainDf["downgrade"]
)

In [12]:
# post balancing check
# print value counts downgrade
print(balancedTrainDfY.value_counts())

downgrade
False    77752
True     15878
Name: count, dtype: int64


### <u>**Step 4**</u>: Regularization / Normalization
some blurb about scalers  

1. [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)             
2. [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html)  
3. [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)  
4. [RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)  
5. [Normalizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html)  
6. [PowerTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html)  
7. [QuantileTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html)  

In [13]:
def printMetrics(model_name, y_true, y_pred):
    print(model_name)
    print("Accuracy: ", accuracy_score(y_true, y_pred))
    print("Precision: ", precision_score(y_true, y_pred))
    print("Recall: ", recall_score(y_true, y_pred))
    print("F1: ", f1_score(y_true, y_pred))
    print("ROC AUC: ", roc_auc_score(y_true, y_pred))
    print("Classification Report: \n", classification_report(y_true, y_pred))
    print()

### <u>**Step 5**</u>: Gradient Boosting Classifier Model

##### <u>**Step 5.1**</u>: Initialize the model

In [14]:
ESTIMATORS = 400
DEPTH = 40
CORES = -1
MINSPLSPLIT = 8
MINSAMPLELEAF = 4

gradient_boosting_model = GradientBoostingClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    verbose=1,
    n_iter_no_change=200,
)
balanced_gradient_boosting_model = GradientBoostingClassifier(
    n_estimators=ESTIMATORS,
    random_state=42,
    max_depth=DEPTH,
    verbose=1,
    n_iter_no_change=200,
)
rusboost_model = RUSBoostClassifier(
    n_estimators=ESTIMATORS, random_state=42, sampling_strategy=0.5
)
balanced_rusboost_model = RUSBoostClassifier(
    n_estimators=ESTIMATORS, random_state=42, sampling_strategy=0.5
)
xgboost_model = XGBClassifier(
    n_estimators=ESTIMATORS, random_state=42, max_depth=DEPTH, verbosity=1, n_jobs=CORES
)
balanced_xgboost_model = XGBClassifier(
    n_estimators=ESTIMATORS, random_state=42, max_depth=DEPTH, verbosity=1, n_jobs=CORES
)

##### <u>**Step 5.2**</u>: Fit the training data to the model

In [15]:
gradient_boosting_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_gradient_boosting_model.fit(balancedTrainDfX, balancedTrainDfY)

      Iter       Train Loss   Remaining Time 
         1           0.1921          782.22m
         2           0.1861          776.89m
         3           0.1819          775.55m
         4           0.1786          766.19m
         5           0.1761          753.05m
         6           0.1739          687.79m
         7           0.1722          627.68m
         8           0.1707          588.95m
         9           0.1695          534.67m
        10           0.1684          489.71m
        20           0.1630          286.10m
        30           0.1614          215.93m
        40           0.1608          178.71m
        50           0.1607          157.61m
        60           0.1606          139.41m
        70           0.1606          125.12m
        80           0.1606          113.72m
        90           0.1605          104.54m
       100           0.1605           96.61m


In [ ]:
rusboost_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_rusboost_model.fit(balancedTrainDfX, balancedTrainDfY)

RUSBoostClassifier(n_estimators=400, random_state=42, sampling_strategy=0.5)

In [ ]:
xgboost_model.fit(trainDf.drop(columns="downgrade"), trainDf["downgrade"])
balanced_xgboost_model.fit(balancedTrainDfX, balancedTrainDfY)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:present_prev1: object, present_prev2: object, present_prev3: object

##### <u>**Step 5.3**</u>: Test the model on the testing dataset

In [ ]:
# set nan to 0
testDf = testDf.fillna(0)

In [ ]:
# get predictions

predictions_gradient_boosting = gradient_boosting_model.predict(
    testDf.drop(columns="downgrade")
)
predictions_balanced_gradient_boosting = balanced_gradient_boosting_model.predict(
    testDf.drop(columns="downgrade")
)
predictions_rusboost = rusboost_model.predict(testDf.drop(columns="downgrade"))
predictions_balanced_rusboost = balanced_rusboost_model.predict(
    testDf.drop(columns="downgrade")
)
# predictions_xgboost = xgboost_model.predict(testDf.drop(columns="downgrade"))
# predictions_balanced_xgboost = balanced_xgboost_model.predict(
#     testDf.drop(columns="downgrade")
# )

In [ ]:
print(pd.DataFrame(predictions_gradient_boosting).value_counts())
print(pd.DataFrame(predictions_balanced_gradient_boosting).value_counts())
print(pd.DataFrame(predictions_rusboost).value_counts())
print(pd.DataFrame(predictions_balanced_rusboost).value_counts())
# print(pd.DataFrame(predictions_xgboost).value_counts())
# print(pd.DataFrame(predictions_balanced_xgboost).value_counts())

False    27323
Name: count, dtype: int64
False    20846
True      6477
Name: count, dtype: int64
False    21013
True      6310
Name: count, dtype: int64
False    26160
True      1163
Name: count, dtype: int64


##### <u>**Step 5.4**</u>: Evaluate models based on different metrics:
- ACCURACY:
- PRECISION:
- RECALL:
- F1:
- ROC AUC:

In [ ]:
# get accuracy precision recall f1 roc_auc
printMetrics(
    "sk GB imbalanced train set", testDf["downgrade"], predictions_gradient_boosting
)
printMetrics(
    "imb GB balanced train set",
    testDf["downgrade"],
    predictions_balanced_gradient_boosting,
)
printMetrics("sk RUS imbalanced train set", testDf["downgrade"], predictions_rusboost)
printMetrics(
    "imb RUS balanced train set", testDf["downgrade"], predictions_balanced_rusboost
)
# printMetrics("sk XGB imbalanced train set", testDf["downgrade"], predictions_xgboost)
# printMetrics(
#     "imb XGB balanced train set", testDf["downgrade"], predictions_balanced_xgboost
# )

sk GB imbalanced train set
Accuracy:  0.9628152106284082
Precision:  0.0
Recall:  0.0
F1:  0.0
ROC AUC:  0.5
Classification Report: 
               precision    recall  f1-score   support

       False       0.96      1.00      0.98     26307
        True       0.00      0.00      0.00      1016

    accuracy                           0.96     27323
   macro avg       0.48      0.50      0.49     27323
weighted avg       0.93      0.96      0.94     27323


imb GB balanced train set
Accuracy:  0.7387914943454232
Precision:  0.027481858885286398
Recall:  0.17519685039370078
F1:  0.047511010276257835
ROC AUC:  0.467877438387256
Classification Report: 
               precision    recall  f1-score   support

       False       0.96      0.76      0.85     26307
        True       0.03      0.18      0.05      1016

    accuracy                           0.74     27323
   macro avg       0.49      0.47      0.45     27323
weighted avg       0.93      0.74      0.82     27323


sk RUS imbala

/home/student/maith/CGC_Grain_Outcome_Predictions/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/student/maith/CGC_Grain_Outcome_Predictions/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/student/maith/CGC_Grain_Outcome_Predictions/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

##### <u>**Step 5.5**</u>: Exports metric results to csv